# Time-Series Forecasting with Spiking Neural Network

## 1. Setup and Imports

Source `.venv/bin/python` as Jupiter Notebook kernel.

## 2. Load Configuration

## 3. Data Loading

## 4. Data Visualization

## 5. Data Preprocessing

## 6. Model Definition

## 7. Training

## 8. Training Visualization

## 9. Evaluation

## 10. Results Visualization

## 11. Save Results